In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204.2 MB 32 kB/s 
     |████████████████████████████████| 198 kB 46.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612244 sha256=27560dda8bc360cc1d0d9b4abc9bafc5090f7d4e430bec441303a748bc647009
  Stored in directory: /root/.cache/pip/wheels/ea/21/84/970b03913d0d6a96ef51c34c878add0de9e4ecbb7c764ea21f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("House Linear Regression Example") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2022-12-15 12:25:23--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.209.81, 52.92.131.74, 52.92.177.34, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.209.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.08s   

2022-12-15 12:25:24 (435 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [ ]:
!ls -tl

total 40
drwxr-xr-x 1 root root  4096 Dec 13 14:35 sample_data
-rw-r--r-- 1 root root 36240 Jan 31  2021 boston_housing.csv


In [ ]:
data = spark.read.csv("./boston_housing.csv", header = True, inferSchema = True)
data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [ ]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

### feature vector 생성

In [ ]:
from pyspark.ml.feature import VectorAssembler #데이터기반 머신러닝 라이브러리 

feature_columns = data.columns[:-1] #레이블을 제외해야 하므로 마지막 column은 제외
assembler = VectorAssembler(inputCols= feature_columns, outputCol = "features")

In [ ]:
data_2 = assembler.transform(data)

In [ ]:
data_2.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

scalerModel = scaler.fit(data_2)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(data_2)

In [ ]:
scaledData.select('scaledFeatures').show(5)

+--------------------+
|      scaledFeatures|
+--------------------+
|[7.34751712931429...|
|[0.00317501096205...|
|[0.00317268579840...|
|[0.00376327736512...|
|[0.00802762749650...|
+--------------------+
only showing top 5 rows



훈련/테스트 데이터를 나누고 모델을 만듦

In [ ]:
train, test = scaledData.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol = "scaledFeatures", labelCol = "medv")
model = algo.fit(train)

모델 성능 측정

In [ ]:
evaluation_summary = model.evaluate(test)

print(f"meanAbsoluteError {evaluation_summary.meanAbsoluteError}")
print(f"rootMeanSquaredError {evaluation_summary.rootMeanSquaredError}")

meanAbsoluteError 3.6336300167332602
rootMeanSquaredError 5.367165521366176


모델 예측값 살펴보기

In [ ]:
predictions = model.transform(test)
predictions.select('medv', 'prediction').show()

+----+------------------+
|medv|        prediction|
+----+------------------+
|24.0|30.126309817765073|
|32.2| 31.33764643197701|
|35.4|31.089607296066887|
|50.0|40.344231748721846|
|29.1|31.186374853148365|
|50.0|44.336400750452945|
|44.0| 36.97587898281213|
|30.1|  25.7557647714603|
|30.8| 30.84169067400308|
|25.0| 28.96296458556564|
|18.5|19.684533211581545|
|34.9|29.903383499787783|
|19.4| 23.44781286009937|
|28.5| 32.84796840982423|
|35.4|34.149623546612176|
|28.0|28.511603940394856|
|22.9| 28.81749998217859|
|20.6|26.747579156845497|
|24.8| 30.40445252617697|
|22.6|27.046206197022784|
+----+------------------+
only showing top 20 rows



구글 드라이브 연결해서 model save

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
model.save("boston_housing_model")

In [ ]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load(model_saved_path)
new_predictions = loaded_model.transform(test)